In [6]:
# Install and import
import os, sys, numpy as np
#from osgeo import gdal
from numpy import flatnonzero as find
from glob import glob
try: # Check platform (Colab or Jupyter)
  import google.colab
  my_drive = "/content/drive/My Drive/"
except:
  my_drive = os.path.expanduser("~") + "/drive/"
path = my_drive + "jöklar/"
sys.path.append(path + "src")
from util.util import install_import
rasterio = install_import("rasterio")

In [7]:
# Define data to use
datafolder = "jöklar/data"
dataname = "lang"
datapath = path + "data/" + dataname + "/"

In [8]:
# Define functions
def read_tile(datapath, n):
    #dataname = os.path.basename(datapath.rstrip('/'))
    imgfile = datapath + f"/img-{n:03d}.tif"
    maskfile = datapath + f"/mask-{n:03d}.tif"
    with rasterio.open(imgfile) as f:
        img_channels = f.meta["count"]
        img = np.array([f.read(b) for b in range(1, img_channels + 1)])
        x = f.meta["transform"][2]
        y = f.meta["transform"][5]
    with rasterio.open(maskfile) as f:
        mask_channels = f.meta["count"]
        masks = np.array([f.read(b) for b in range(1, mask_channels + 1)])
        mask = np.any(masks, axis=0)
    return img, mask, x, y

In [14]:
img, mask, x, y = read_tile(datapath, 21)
img.dtype, mask.dtype

(dtype('uint16'), dtype('bool'))

In [17]:
def read_tiles(datapath):
    # Read a whole glacier
    nfiles = len(glob(f"{datapath}/img-*"))
    img = nfiles*[None]
    mask = np.zeros((nfiles, 256, 256), bool)
    x = np.zeros(nfiles)
    y = np.zeros(nfiles)
    for n in range(nfiles):
        img[n], mask[n], x[n], y[n] = read_tile(datapath, n)
        if n % 10 == 0:
            print(f"{n}, ", end="")
    print()
    col = ((x - x.min())/2560).astype(int)
    row = ((y - y.min())/2560).astype(int)
    I = np.lexsort((col, row))
    row = row[I]
    col = col[I]
    mask = mask[I]
    data = np.transpose(np.array(img)[I], (0, 2, 3, 1))/10000
    return data, mask, row, col

def print_info(image, mask, row, col):
    # Print summary and tif-meta information
    w = (col[-1] + 1)*2.56
    h = (row[-1] + 1)*2.56
    pct_glacier = np.sum(mask)/np.size(mask)*100
    print()
    print(f"Image shape: {image.shape}, data type: {image.dtype}")
    print(f"Region width {w} km, height {h} km")
    print(f"Pixels labeled glacier: {pct_glacier:.1f}%")
    print()

def save_data_and_report(outfile, image, mask, row, col):
    np.savez_compressed(outfile, image=image, mask=mask, row=row, col=col)
    size = os.path.getsize(outfile)
    print(size)
    compress_ratio = size/(image.nbytes + mask.nbytes + row.nbytes + col.nbytes)
    print(f"Wrote {size/2**20:.1f} Mb of data to {outfile}")
    print(f"  Compression ratio: {compress_ratio:.2f}")
    print()

In [ ]:
# Read data and report
image, mask, row, col = read_tiles(datapath)
print_info(image, mask, row, col)

In [23]:
image.nbytes/1e9, mask.nbytes/1e9

(1.342701568, 0.012910592)

In [ ]:
# Save to compressed file and report
dataname = os.path.basename(datapath.rstrip("/"))
outfile = datapath + dataname + ".npz"
save_data_and_report(outfile, image, mask, row, col)

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 

Image shape: (197, 256, 256, 13), data type: float64
Region width 48.64 km, height 48.64 km
Pixels labeled glacier: 68.5%

229518533
Wrote 218.9 Mb of data to /Users/jonasson/drive/jöklar/data/lang/lang.npz
  Compression ratio: 0.17



In [ ]:
%%time
# Test reading of the file
print(outfile)
data = np.load(outfile)
[img, mask, row, col] = data.values()
img.shape, mask.shape, row.shape, col.shape